In [1]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
from scipy.stats import pearsonr
from sklearn.metrics import roc_auc_score

In [3]:
# load dataset
data_in = 'all_z_nonCenter_p001_20190530'
dataset = data_in + '_500PCs'
npc=500
z = np.load('/oak/stanford/groups/mrivas/projects/degas-risk/datasets/all_pop/'+dataset+'.npz')
scores = pd.read_table('/oak/stanford/groups/mrivas/projects/degas-risk/scorefiles/'+dataset+'_full.profile',
                       sep='\s+',
                       index_col='IID').sort_index()
scores.iloc[:,:10].tail()

,SCORE_PC1,SCORE_PC2,SCORE_PC3,SCORE_PC4,SCORE_PC5,SCORE_PC6,SCORE_PC7,SCORE_PC8,SCORE_PC9,SCORE_PC10
IID,,,,,,,,,,
6026191,3338.130,2182.780,2013.27,1641.860,-2818.540,-2449.70,2455.34,507.033,863.724,662.515
6026202,-222.339,-585.984,-1583.68,-580.076,3556.120,3214.89,-1881.74,616.321,-423.946,1101.010
6026216,5595.960,1424.090,-1767.66,-911.971,562.812,2280.03,-1941.71,728.371,-1424.610,-258.517
6026229,302.365,-1000.650,1026.43,-474.952,129.932,-3920.62,1425.93,-523.837,204.433,-380.698
6026237,-790.597,2918.800,1479.87,1495.970,-601.537,-3141.84,1167.40,-2669.030,829.562,1262.550


In [4]:
# load PRS data (includes phenotypes!)
phenos=['INI50','INI21001','HC269','HC382','HC221','HC294','HC326','cancer1060']

# helper
prs='all_beta_center_p001_20190530'
get_f = lambda phe: '/oak/stanford/groups/mrivas/projects/degas-risk/PRS/all_pop/'+prs+'/'+phe+'_PRS.profile'
# load
data = reduce(lambda x,y: pd.merge(x, y, left_index=True, right_index=True), 
              map(lambda phe: pd.read_table(get_f(phe), 
                                            sep='\s+', 
                                            skiprows=1,
                                            na_values=-9,
                                            usecols=[1,2,5],
                                            names=['IID',phe,phe+'_PRS'],
                                            index_col='IID'), 
                  phenos)).sort_index()

# binary traits are 1/2, they need to be 0/1
data[[i for i in data.columns if 'INI' not in i]] -= 1

# add covariates
data = data.merge(pd.read_table('/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/master_phe/master.phe',
                                usecols=['IID','age','sex','PC1','PC2','PC3','PC4'], 
                                index_col='IID'), 
                  left_index=True, right_index=True)

# show
data.tail()

,INI50,INI50_PRS,INI21001,INI21001_PRS,HC269,HC269_PRS,HC382,HC382_PRS,HC221,HC221_PRS,...,HC326,HC326_PRS,cancer1060,cancer1060_PRS,age,sex,PC1,PC2,PC3,PC4
IID,,,,,,,,,,,,,,,,,,,,,
6026191,162.0,81.2988,32.5789,94.27250,0.0,-32.03870,0.0,3.58478,0.0,-20.0895,...,0.0,9.06200,0.0,-28.22480,67.0,0,-10.3399,1.73064,-0.731337,-2.44991
6026202,168.0,29.2941,24.1993,-14.45570,0.0,3.55312,0.0,24.51580,0.0,36.1575,...,0.0,4.10299,0.0,13.64210,54.0,1,-10.7368,4.90997,-0.799375,-4.69019
6026216,183.0,46.9242,33.6827,119.70100,0.0,-33.27100,0.0,-51.42370,0.0,31.8385,...,0.0,11.56950,0.0,12.82230,56.0,1,-12.5254,3.52680,-2.443390,2.61381
6026229,174.0,19.6134,25.4657,-7.34802,0.0,-25.15460,0.0,13.68410,0.0,-30.1255,...,0.0,12.17560,0.0,8.35452,77.0,1,-13.4920,3.33105,-0.616797,10.03480
6026237,180.0,-80.7357,32.5617,50.60180,1.0,-8.52000,0.0,47.50080,1.0,29.6865,...,1.0,-4.43399,0.0,28.21620,77.0,1,-15.2664,5.98740,-3.865480,5.89699


In [5]:
# add DEGAS Risk score to data
for phe in phenos:
    pcvec = z['V'][np.where(z['label_phe_code'] == phe),:].flatten()[:npc]
    data[phe+'_DEGAS_{}'.format(npc)] = scores.iloc[:,:npc].dot(pcvec)
data.tail()

,INI50,INI50_PRS,INI21001,INI21001_PRS,HC269,HC269_PRS,HC382,HC382_PRS,HC221,HC221_PRS,...,PC3,PC4,INI50_DEGAS_500,INI21001_DEGAS_500,HC269_DEGAS_500,HC382_DEGAS_500,HC221_DEGAS_500,HC294_DEGAS_500,HC326_DEGAS_500,cancer1060_DEGAS_500
IID,,,,,,,,,,,,,,,,,,,,,
6026191,162.0,81.2988,32.5789,94.27250,0.0,-32.03870,0.0,3.58478,0.0,-20.0895,...,-0.731337,-2.44991,496.618993,893.557767,46.610692,61.464837,46.821384,-1.403974,20.038773,17.881528
6026202,168.0,29.2941,24.1993,-14.45570,0.0,3.55312,0.0,24.51580,0.0,36.1575,...,-0.799375,-4.69019,238.155358,-202.247354,19.913321,85.158123,85.182974,12.661987,3.288283,7.183828
6026216,183.0,46.9242,33.6827,119.70100,0.0,-33.27100,0.0,-51.42370,0.0,31.8385,...,-2.443390,2.61381,131.410663,1109.153944,16.645426,8.983150,69.720009,8.920991,16.425397,-9.291838
6026229,174.0,19.6134,25.4657,-7.34802,0.0,-25.15460,0.0,13.68410,0.0,-30.1255,...,-0.616797,10.03480,430.726301,-349.160133,-34.424074,-75.366079,-82.306273,-8.166792,-5.494142,-10.388817
6026237,180.0,-80.7357,32.5617,50.60180,1.0,-8.52000,0.0,47.50080,1.0,29.6865,...,-3.865480,5.89699,-658.089042,302.254373,18.638476,60.286731,45.284303,2.145960,15.656680,4.515551


In [6]:
# populations for evaluation (white british is in sample, non-british white is external)
a,b = 'white_british', 'non_british_white'
with open('/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification/ukb24983_'+a+'.phe', 'r') as f:
    wb = set(filter(lambda x:x in data.index, [int(line.split()[0]) for line in f]))

with open('/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification/ukb24983_'+b+'.phe', 'r') as f:
    nbw = set(filter(lambda x:x in data.index, [int(line.split()[0]) for line in f]))
    
print(len(wb), len(nbw))

(337151, 25486)


In [7]:
# helper -- AUC for bins, r^2 on residual after regressing out age/sex for QTs
def score(pheno, model, pop):
    if 'INI' in pheno:
        pdata = ols(pheno+'~age+sex', data.loc[pop,:]).fit().resid
        x,xhat= pdata.dropna().align(data.loc[pop,pheno+'_'+m].dropna(), join='inner')
        return pearsonr(x,xhat)[0]**2
    else:
        return roc_auc_score(data.loc[pop,pheno].astype(bool), data.loc[pop,pheno+'_'+model])

In [8]:
# evaluate  
models = ['PRS','DEGAS_'+str(npc)]

# within sample
matt = pd.DataFrame([[score(p,m,wb) for m in models] for p in phenos], columns=models, index=phenos)
matt['CORR'] = matt.index.map(lambda s: data.loc[wb,[s+'_'+m for m in models]].corr().iloc[0,1]**2)

# out of sample
matt2 = pd.DataFrame([[score(p,m,nbw) for m in models] for p in phenos], columns=models, index=phenos)
matt2['CORR'] = matt.index.map(lambda s: data.loc[nbw,[s+'_'+m for m in models]].corr().iloc[0,1]**2)

In [9]:
# show
pd.concat((matt,matt2), keys=['WB','NBW'])

PRS  DEGAS_500      CORR
WB  INI50       0.236268   0.203848  0.964732
    INI21001    0.147771   0.122463  0.957803
    HC269       0.722799   0.552529  0.360479
    HC382       0.641084   0.548352  0.581137
    HC221       0.730077   0.562212  0.654079
    HC294       0.915415   0.550969  0.018636
    HC326       0.869517   0.574445  0.047693
    cancer1060  0.775260   0.544809  0.083040
NBW INI50       0.200110   0.190669  0.959398
    INI21001    0.054407   0.051713  0.954229
    HC269       0.590892   0.550644  0.375458
    HC382       0.572641   0.535568  0.566665
    HC221       0.586353   0.550891  0.658960
    HC294       0.516455   0.550622  0.020021
    HC326       0.566767   0.543982  0.046278
    cancer1060  0.598671   0.527229  0.078470